<a href="https://colab.research.google.com/github/Bo-Ni/ProtBert_Notebook_set_0/blob/main/DSSP_Predict_SS_0_prepare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. install miniconda for google clab
### Run this part ONLY ONCE

In [5]:
# ref: https://pypi.org/project/condacolab/
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:29
🔁 Restarting kernel...


In [1]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


### 1. prepare the packages

In [2]:
# prepare the backages for the environment
import os
import sys

In [3]:
IF_Google = 1

In [20]:
if IF_Google == 1:
  try:
    from Bio.PDB.DSSP import DSSP
  except ImportError as e:
    # print(os.popen('conda install -c salilab dssp').read()) 
    # this one didn't work; Seem the version doesn't match

    print(os.popen('conda install -c ostrokach dssp').read())
    print(os.popen('conda install -c conda-forge biopython').read())
    # adding more here next time


In [21]:
print(os.popen('conda install -c ostrokach dssp').read())

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - dssp


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    dssp-2.2.1                 |                1         4.1 MB  ostrokach
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

  dssp               ostrokach/linux-64::dssp-2.2.1-1




dssp-2.2.1           | 4.1 MB    |            |   0% 
dssp-2.2.1           | 4.1 MB    | ########## | 100% 
dssp-2.2.1           | 4.1 MB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done



### 2. Real subroutine begin here

#### Task 1: calculate secondary structure given a pdb, testing against PDB and AF_PDB files

In [5]:
# 1. download the pdb files
if IF_Google == 1:
  file_exists = os.path.exists('./AF-Q47125-F1-model_v2.pdb')
  if not (file_exists):
    # download files
    # AF-PDB: https://alphafold.ebi.ac.uk/files/AF-Q47125-F1-model_v2.pdb
    print(os.popen('wget https://alphafold.ebi.ac.uk/files/AF-Q47125-F1-model_v2.pdb').read())
    # PDB: https://files.rcsb.org/download/4BWD.pdb
    print(os.popen('wget https://files.rcsb.org/download/4BWD.pdb').read())
    # with chain break
    print(os.popen('wget https://files.rcsb.org/download/1YYJ.pdb').read())


In [137]:
# print(os.popen('wget https://files.rcsb.org/download/1YYJ.pdb').read())

In [36]:
# 2. process pdb files: 4BWD.pdb: A,B,C chains
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP
import Bio.PDB as PDB

In [91]:
import numpy as np

In [138]:
p = PDBParser()
structure_1 = p.get_structure("4BWD", "./4BWD.pdb")
structure_2 = p.get_structure("AF-Q47125", "./AF-Q47125-F1-model_v2.pdb")
structure_3 = p.get_structure("1YYJ", "./1YYJ.pdb")

/usr/local/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 1795.
  PDBConstructionWarning,
/usr/local/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 1815.
  PDBConstructionWarning,
/usr/local/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 1834.
  PDBConstructionWarning,


In [15]:
model_1_s1 = structure_1[0]
for chain in model_1_s1:
    print(chain)

<Chain id=A>
<Chain id=B>
<Chain id=C>


In [50]:
print( chain.full_id )
print( chain.full_id[2] )
print( len(model_1_s1) )

('4BWD', 0, 'C')
C
3


In [53]:
# find all the chains
def count_chains_in_model(model):
  Chain_List=''
  N_Chain = len(model)
  for chain in model:
    Chain_List += chain.full_id[2]
  return Chain_List,N_Chain


In [54]:
ChainList,NChain = count_chains_in_model(model_1_s1)
print(ChainList)
print(NChain)

ABC
3


In [18]:
print(model_1_s1['A'])

<Chain id=A>


In [27]:
dssp_1_s1 = DSSP(model_1_s1, "./4BWD.pdb")

In [94]:
# extract sequence and ss from the dssp tuple
def get_AA_SS_from_DSSP(dssp_1_s1):
  AA_Full = np.array([]) # ''
  SS_Full = np.array([]) # ''
  ChainID_Full = np.array([]) # ''

  for z in range(len(dssp_1_s1)):
    a_key = list(dssp_1_s1.keys())[z]
    # ChainID_Full += a_key[0]
    # AA_Full += dssp_1_s1[a_key][1]
    # SS_Full += dssp_1_s1[a_key][2]
    # for np
    ChainID_Full = np.append( ChainID_Full,a_key[0] )
    AA_Full = np.append( AA_Full, dssp_1_s1[a_key][1] )
    SS_Full = np.append( SS_Full, dssp_1_s1[a_key][2] )
  return ChainID_Full,AA_Full,SS_Full


In [135]:
# divide the result records according to the chains
def Divide_Result_For_Chains(ChainID_Full,AA_Full,SS_Full):
  UniChainList = ''.join(set(ChainID_Full))
  NChain=len(UniChainList)

  resu={"Chain":[],"AA":[],"SS":[]}
  for ThisChain in UniChainList:
    id_pick=ChainID_Full==ThisChain
    This_AA=AA_Full[id_pick]
    This_SS=SS_Full[id_pick]
    # debug
    # print(This_AA)
    resu["Chain"].append(ThisChain)
    # resu["AA"].append(["".join(item) for item in This_AA.astype(str)])
    resu["AA"].append(''.join(map(str, This_AA)))
    resu["SS"].append(''.join(map(str, This_SS)))
  
  return NChain,resu

In [136]:
ChainID_Full,AA_Full,SS_Full=get_AA_SS_from_DSSP(dssp_1_s1)
# they are np arrays
print(ChainID_Full)
print(AA_Full)
print(SS_Full)
NChain, Resu_Dic = Divide_Result_For_Chains(ChainID_Full,AA_Full,SS_Full)
print(Resu_Dic)

['A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'C' 'C' 'C' 'C' 'C'
 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C'
 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C'
 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C'
 'C' 'C']
['V' 'D' 'A' 'E' 'N' 'Q' 'V' 'E' 'L' 'E' 'E' 'K' 'T' 'R' 'L' 'I' 'N' 'Q'
 'V' 'M' 'E' 'L' 'Q' 'H' 'T' 'L' 'E' 'D' 'L' 'S' 'A' 'R' 'V' 'D' 'A' 'V'
 'K' 'E' 'E' 'N' 'L' 'K' 'L' 'K' 'S' 'E' 'N' 'Q' 'V' 'L' 'G' 'Q' 'Y' 'I'
 'E' 'N' 'L' 'M' 'S' 'A' 'S' 'A' 'E' 'N' 

In [140]:
model_3_s1 = structure_3[0]
dssp_3_s1 = DSSP(model_3_s1, "./1YYJ.pdb")

ChainID_Full,AA_Full,SS_Full=get_AA_SS_from_DSSP(dssp_3_s1)
# they are np arrays
print(ChainID_Full)
print(AA_Full)
print(SS_Full)
NChain, Resu_Dic = Divide_Result_For_Chains(ChainID_Full,AA_Full,SS_Full)
print(Resu_Dic)
print(NChain)

['A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A']
['A' 'D' 'L' 'E' 'D' 'N' 'W' 'E' 'T' 'L' 'N' 'D' 'N' 'L' 'K' 'V' 'I' 'E'
 'K' 'A' 'D' 'N' 'A' 'A' 'Q' 'V' 'K' 'D' 'A' 'L' 'T' 'K' 'M' 'R' 'A' 'A'
 'A' 'L' 'D' 'A' 'Q' 'K' 'A' 'T' 'P' 'P' 'K' 'L' 'E' 'D' 'K' 'S' 'P' 'D'
 'S' 'P' 'E' 'M' 'K' 'D' 'F' 'R' 'H' 'G' 'F' 'D' 'I' 'L' 'V' 'G' 'Q' 'I'
 'D' 'D' 'A' 'L' 'K' 'L' 'A' 'G' 'K' 'V' 'K' 'E' 'A' 'Q' 'A' 'A' 'A' 'E'
 'Q' 'L' 'K' 'T' 'T' 'I' 'R' 'A' 'Y' 'N' 'Q' 'K' 'Y' 'G']
['-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
 'T' 'T' '-' 'S' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
 'H' 'H'

In [141]:
model_2_s1 = structure_2[0]
dssp_2_s1 = DSSP(model_2_s1, "./AF-Q47125-F1-model_v2.pdb")

ChainID_Full,AA_Full,SS_Full=get_AA_SS_from_DSSP(dssp_2_s1)
# they are np arrays
print(ChainID_Full)
print(AA_Full)
print(SS_Full)
NChain, Resu_Dic = Divide_Result_For_Chains(ChainID_Full,AA_Full,SS_Full)
print(Resu_Dic)
print(NChain)

['A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A

In [100]:
# divide the results according to chains from DSSP
print(list(set(ChainID_Full)))
print(''.join(set(ChainID_Full)))


# AA_Full[0:2]

# def find_char_index_in_string(s, ch):
#     return [i for i, ltr in enumerate(s) if ltr == ch]

# pick_id = find_char_index_in_string(ChainID_Full,'A')
# print(pick_id)

# ChainID_Full[pick_id]

# ChainID_Full[ ChainID_Full=='A' ]
#print(ChainID_Full. [pick_id])
# print(ChainID_Full[pick_id])
#ChainID_Full[:]=='A'
#ChainID_Full.finditer('A')
# AA_Full[]

['C', 'B', 'A']
CBA


In [128]:
resu["SS"][0]

'--SHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH-'

In [46]:
# if only select Chain A or the first chain
AA_ChainA = ''
SS_ChainA = ''


for z in range(len(dssp_1_s1)):
  a_key = list(dssp_1_s1.keys())[z]
  if a_key[0]=='A':
    AA_ChainA += dssp_1_s1[a_key][1]
    SS_ChainA += dssp_1_s1[a_key][2]

print(AA_ChainA)
print(SS_ChainA)

VDAENQVELEEKTRLINQVMELQHTLEDLSARVDAVKEENLKLKSENQVLGQYIENLMSAS
-TTTHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHT-


In [45]:
a_key = list(dssp_1_s1.keys())[0]
print(a_key)
print(a_key[1])

('A', (' ', 86, ' '))
(' ', 86, ' ')


In [34]:
dssp_1_s1[('A',(' ',86,' '))]

(1,
 'V',
 '-',
 0.8380281690140845,
 360.0,
 -72.6,
 0,
 0.0,
 4,
 -0.5,
 0,
 0.0,
 0,
 0.0)

In [28]:
dssp_1_s1.property_list[0]

(1,
 'V',
 '-',
 0.8380281690140845,
 360.0,
 -72.6,
 0,
 0.0,
 4,
 -0.5,
 0,
 0.0,
 0,
 0.0)

In [23]:
dssp_1_s1.keys()

[('A', (' ', 86, ' ')),
 ('A', (' ', 87, ' ')),
 ('A', (' ', 88, ' ')),
 ('A', (' ', 89, ' ')),
 ('A', (' ', 90, ' ')),
 ('A', (' ', 91, ' ')),
 ('A', (' ', 92, ' ')),
 ('A', (' ', 93, ' ')),
 ('A', (' ', 94, ' ')),
 ('A', (' ', 95, ' ')),
 ('A', (' ', 96, ' ')),
 ('A', (' ', 97, ' ')),
 ('A', (' ', 98, ' ')),
 ('A', (' ', 99, ' ')),
 ('A', (' ', 100, ' ')),
 ('A', (' ', 101, ' ')),
 ('A', (' ', 102, ' ')),
 ('A', (' ', 103, ' ')),
 ('A', (' ', 104, ' ')),
 ('A', (' ', 105, ' ')),
 ('A', (' ', 106, ' ')),
 ('A', (' ', 107, ' ')),
 ('A', (' ', 108, ' ')),
 ('A', (' ', 109, ' ')),
 ('A', (' ', 110, ' ')),
 ('A', (' ', 111, ' ')),
 ('A', (' ', 112, ' ')),
 ('A', (' ', 113, ' ')),
 ('A', (' ', 114, ' ')),
 ('A', (' ', 115, ' ')),
 ('A', (' ', 116, ' ')),
 ('A', (' ', 117, ' ')),
 ('A', (' ', 118, ' ')),
 ('A', (' ', 119, ' ')),
 ('A', (' ', 120, ' ')),
 ('A', (' ', 121, ' ')),
 ('A', (' ', 122, ' ')),
 ('A', (' ', 123, ' ')),
 ('A', (' ', 124, ' ')),
 ('A', (' ', 125, ' ')),
 ('A', (' ', 1

In [12]:
len( structure_1 )
structure_1[0]

<Model id=0>

In [4]:
!conda install -c ostrokach dssp

Solving environment: \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - dssp


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.10.8  |       ha878542_0         139 KB  conda-forge
    certifi-2021.10.8          |   py37h89c1867_2         145 KB  conda-forge
    conda-4.12.0               |   py37h89c1867_0         1.0 MB  conda-forge
    dssp-2.2.1                 |                1         4.1 MB  ostrokach
    ------------------------------------------------------------
                                           Total:         5.3 MB

The following NEW packages will be INSTALLED:

  dssp               ostrokach/linux-64::dssp-2.2.1-1

The following packages will be UPDATED:

  ca-certificates                      2020.12.5-ha878542_0 --> 2021.10.8-ha878542_0
  certifi                          2020.12

In [5]:
!conda install -c conda-forge biopython

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - biopython


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    biopython-1.79             |   py37h5e8e339_0         2.6 MB  conda-forge
    libblas-3.9.0              |11_linux64_openblas          12 KB  conda-forge
    libcblas-3.9.0             |11_linux64_openblas          11 KB  conda-forge
    libgfortran-ng-11.2.0      |      h69a702a_16          23 KB  conda-forge
    libgfortran5-11.2.0        |      h5c6108e_16         1.7 MB  conda-forge
    liblapack-3.9.0            |11_linux64_openblas          11 KB  conda-forge
    libopenblas-0.3.17         |pthreads_h8fe5266_1         9.2 MB  conda-forge
    numpy-1.20.3               |   py37h038b26d_1         5.7 MB  conda-forg

In [7]:
# test BioPython
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP

In [8]:
# add the pdb files

In [9]:
p = PDBParser()
structure = p.get_structure("1MOT", "./1mot.pdb")
model = structure[0]
dssp = DSSP(model, "./1mot.pdb")

In [10]:
dssp.keys()

[('A', (' ', 249, ' ')),
 ('A', (' ', 250, ' ')),
 ('A', (' ', 251, ' ')),
 ('A', (' ', 252, ' ')),
 ('A', (' ', 253, ' ')),
 ('A', (' ', 254, ' ')),
 ('A', (' ', 255, ' ')),
 ('A', (' ', 256, ' ')),
 ('A', (' ', 257, ' ')),
 ('A', (' ', 258, ' ')),
 ('A', (' ', 259, ' ')),
 ('A', (' ', 260, ' ')),
 ('A', (' ', 261, ' ')),
 ('A', (' ', 262, ' ')),
 ('A', (' ', 263, ' ')),
 ('A', (' ', 264, ' ')),
 ('A', (' ', 265, ' ')),
 ('A', (' ', 266, ' ')),
 ('A', (' ', 267, ' ')),
 ('A', (' ', 268, ' ')),
 ('A', (' ', 269, ' ')),
 ('A', (' ', 270, ' ')),
 ('A', (' ', 271, ' ')),
 ('A', (' ', 272, ' ')),
 ('A', (' ', 273, ' ')),
 ('A', (' ', 274, ' ')),
 ('A', (' ', 275, ' ')),
 ('A', (' ', 276, ' '))]

In [25]:
len( dssp.property_list )

28

In [36]:
print( dssp.property_list[0] )
print( dssp.property_list[1] )
print( dssp.property_list[2] )

(1, 'A', '-', 1.0, 360.0, 153.9, 0, 0.0, 2, -0.7, 0, 0.0, 3, -0.1)
(2, 'P', '-', 0.625, -68.1, 106.5, 0, 0.0, 4, -2.7, 0, 0.0, 5, -0.2)
(3, 'A', 'H', 0.7075471698113207, -61.2, -42.4, -2, -0.7, 4, -3.0, 1, -0.2, 5, -0.2)


In [28]:
dssp.property_list[27][2]

'-'

In [21]:
dssp.property_list[0]

(1, 'A', '-', 1.0, 360.0, 153.9, 0, 0.0, 2, -0.7, 0, 0.0, 3, -0.1)

In [11]:
for chain in model:
    print(chain)

<Chain id=A>


In [15]:
chain_A = model['A']


<Chain id=A>


In [16]:
print( chain_A[261] )

<Residue LEU het=  resseq=261 icode= >


In [33]:
dssp.property_list[1]

(2, 'P', '-', 0.625, -68.1, 106.5, 0, 0.0, 4, -2.7, 0, 0.0, 5, -0.2)

In [3]:
# download the file
# 8. download the necessary files for google
if IF_Google == 1:
  file_exists = os.path.exists('./casp12-40-fea21c.pkl')
  if not (file_exists):
    # download files
    print(os.popen('wget https://www.dropbox.com/s/pvper16e3knnk4d/casp12-40-fea21c.pkl').read())
    print(os.popen('wget https://www.dropbox.com/s/wsvu8kokma9uey2/casp11-fea21c-85.pkl').read())
    print(os.popen('wget https://www.dropbox.com/s/hhvb7313hzavwen/casp10-fea21c-123.pkl').read())


In [16]:
import pickle as pkl
import pandas as pd

In [17]:
with open("casp10-fea21c-123.pkl", "rb") as f:
    object = pkl.load(f)
    
df = pd.DataFrame(object)
df.to_csv(r'file.csv')

In [25]:
len(object)
len(object[0][0])
object[0][0]

[[-4,
  -4,
  3,
  7,
  -6,
  -2,
  0,
  -3,
  -3,
  -5,
  -6,
  -3,
  -5,
  -6,
  -4,
  -2,
  -3,
  -6,
  -5,
  -5,
  'D'],
 [-4,
  -3,
  4,
  6,
  -6,
  -2,
  3,
  -4,
  -3,
  -6,
  -6,
  -1,
  -5,
  -6,
  -4,
  -2,
  -2,
  0,
  -5,
  -5,
  'D'],
 [-4,
  -3,
  2,
  7,
  -6,
  -2,
  -1,
  -4,
  -4,
  -6,
  -6,
  3,
  -5,
  -6,
  -3,
  -3,
  -2,
  -7,
  -6,
  -2,
  'D'],
 [-2, -2, 1, 2, -4, -2, 0, -2, -2, 4, 0, 1, -2, -3, -4, 0, 0, -4, -1, 1, 'T'],
 [0, -1, 0, 2, -4, 3, 1, -1, 0, -3, -4, 2, -3, -3, 3, 1, 0, -4, -1, -3, 'G'],
 [1, -1, 1, 1, -3, 1, 1, 0, -2, -1, -1, 1, -2, -3, 2, 1, 0, -4, -2, 0, 'Y'],
 [1, -1, 0, 0, -3, -1, 0, -1, -1, 0, -2, 1, -1, 0, -2, 2, 0, -1, -1, 0, 'L'],
 [0, 1, 1, 2, -4, 3, 2, -3, -1, -3, -4, 1, -3, -3, -1, 1, 1, -5, -2, -3, 'P'],
 [0,
  -5,
  -2,
  -6,
  -1,
  -5,
  -5,
  -6,
  -5,
  2,
  3,
  -5,
  -1,
  -3,
  2,
  -3,
  -3,
  -5,
  -4,
  5,
  'P'],
 [-4,
  -6,
  0,
  -1,
  -7,
  -5,
  -4,
  -5,
  -6,
  -7,
  -6,
  -5,
  -7,
  -8,
  8,
  1,
  -4,
  -8,
  -7,
 

In [6]:
df_1 = pd.read_pickle(r'casp10-fea21c-123.pkl')

In [14]:
df_1[1]

[[7,
  3,
  3,
  3,
  5,
  7,
  7,
  7,
  7,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  5,
  5,
  7,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  5,
  5,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  5,
  5,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  7,
  5,
  5,
  6,
  7,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  6,
  6,
  3,
  3,
  3,
  6,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  6,
  5,
  5,
  5,
  5,
  6,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  7,
  6,
  6,
  6,
  6,
  7,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  6,
  6,
  6,
  2,
  2,
  2,
  2,
  2,
  2,
  7,
  5,
  5,
  6,
  7,
  2,
  2,
  2,
  2,
  2,
  2,
  7,
  6,
  6,
  6,
  7,
  7,
  6,
  7,
  7,
  7,
  7,
  7],
 [7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  5,
  5,
  5,
  7,
  7,
  7,
  6,
  6,
  6,
  7,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  5,
  5,
  5,
  0,
  0,
  0,
  0,
  5,
  5,
  6,
  2,
  2,
  7,
  7,
  7,
  3,
  3,
  3,
  5,
  5,
  5,
  3,